In [3]:
import sys
sys.path.append("/home/dux/")
sys.path.append("/home/dux/surface_sampling/sgmc_surf")

from ase.build import make_supercell
from ase.io import read, write

import catkit
from catkit.gen.adsorption import get_adsorption_sites

from mcmc import mcmc_run
import os
import numpy as np

os.environ["PROJECT_DIR"] = os.getcwd()

In [1]:
# SrTiO3 from HTVS db
import sys
import os
import django
import numpy as np
# from tqdm import tqdm
# import pandas as pd
# import pickle as pkl
sys.path.append('/home/dux/htvs/')
sys.path.append('/home/dux/htvs/djangochem/')

#from imports import *
#from django_imports import *
# from pymatgen.core.periodic_table import Element
# from chemconfigs.vasp.defaults import Magmom
# from ase.visualize import view

# setup the django settings file.  Change this to use the settings file that connects you to your desired database
os.environ["DJANGO_SETTINGS_MODULE"] = "djangochem.settings.orgel"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"  # CAUTION!!! LINE NECESSARY WHEN INTERACTING WITH DATABASE IN JUPYTER NOTEBOOK. DO NOT USE THIS LINE IF YOU ARE PLANNING ON SAVING/REMOVING THINGS FROM THE DATABASE!!!!!

# this must be run to setup access to the django settings and make database access work etc.
django.setup()
# from jobs.models import Job, JobConfig
# from django.contrib.contenttypes.models import ContentType
# from ase.io import write

from pgmols.models import Surface
# from analysis.structure.perovskite import PerovskiteAnalyzer
# from pymatgen.core.periodic_table import Element

unit_slab = Surface.objects.get(id=217485271).as_ase_gratoms()

# single multiplication for upsize
pristine_slab = unit_slab*(2,2,1)

In [5]:
# get absolute adsorption coords
element = 'O'
ads = catkit.gratoms.Gratoms(element)

Use manually edited (ase gui)

In [6]:
filtered_ads_slab = read("SrTiO3_001_2x2_O_filtered_adsorbed_slab.cif")
print(f"{len(filtered_ads_slab)}")
print(f"{len(pristine_slab)}")

92
60


In [8]:
ads_positions = filtered_ads_slab.get_positions()[len(pristine_slab):]
len(ads_positions)

32

Set tags

In [10]:
surface_atoms = pristine_slab.get_surface_atoms()
atoms_arr = np.arange(0, len(pristine_slab))
base_tags = np.int8(np.isin(atoms_arr, surface_atoms)).tolist()

In [13]:
slab = filtered_ads_slab.copy()
num_ads_atoms = len(slab) - len(pristine_slab)
tags = list(base_tags)
tags.extend([2]*num_ads_atoms)
slab.set_tags(tags)

In [15]:
print(slab.get_tags())

[0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


Save to .pkl

In [16]:
import pickle
with open("ase_SrTiO3_001_2x2_O_filtered_ads_2022_0707.pkl", "wb") as f:
    pickle.dump(slab, f)